# Kakao Map API

In [1]:
import pandas as pd
import numpy as np

# A simple, yet elegant HTTP library.
import requests
from urllib.parse import urlparse, quote

# A Fast, Extensible Progress Bar for Python and CLI
from tqdm import tqdm
import time

In [2]:
key_fd = open('config/kakaomaprestapikey.txt', mode='r')
API_KEY = key_fd.read(100)
key_fd.close()

In [3]:
df = pd.read_csv('data/roadapi.csv')
df.shape

(77, 24)

In [4]:
def getCoords(query = ''):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    params = { 
        'query': query
    }
    headers = { "Authorization": "KakaoAK " + API_KEY }
    api = requests.get(url, params=params, headers=headers).json()
    coords = {}
    coords.update({ 'x': float(api['documents'][0]['x']) })
    coords.update({ 'y': float(api['documents'][0]['y']) })
    return coords

In [5]:
df['lat'] = np.nan
df['lon'] = np.nan
df.index = np.arange(0, len(df))
for i in tqdm(df.index):
    try:
        addr = df['roadAddr'][i]
        coords = getCoords(addr)
        df['lat'][i] = coords['y']
        df['lon'][i] = coords['x']
    except Exception as e:
        print(i, e)

100%|██████████| 77/77 [00:03<00:00, 21.53it/s]


In [6]:
df.dropna(subset=['lat', 'lon'], inplace=True)
df.index = np.arange(0, len(df))
df.to_csv('data/kakaomapapi.csv', sep=',', encoding='utf8', index=False)